In [3]:
import sys

sys.path.append(r'C:\repos\ra2ce_multi_network') # path to the package
sys.path.append(r'C:\repos\ra2ce') # path to the package

import geopandas as gpd
import pickle

import pyproj
from networkx import MultiDiGraph
from pathlib import Path
from snkit import Network

from ra2ce.graph.exporters.geodataframe_network_exporter import GeoDataFrameNetworkExporter
from ra2ce.graph.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter
from ra2ce.graph.exporters.network_exporter_base import NetworkExporterBase
from ra2ce.graph.network_config_data.network_config_data import NetworkConfigData, NetworkSection
from ra2ce.graph.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper
from ra2ce.graph.origins_destinations import add_od_nodes
from ra2ce_multi_network.simplify_rail import _nx_to_network, _network_to_nx
import ra2ce.graph.networks_utils as nut

c:\Users\asgarpou\AppData\Local\miniconda3\envs\ra2ce_multi_network_env\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\asgarpou\AppData\Local\Temp\ipykernel_25644\4176253495.py:6: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-up

In [4]:
# Defining ini variables
root_folder = Path(
    r'C:\Users\asgarpou\OneDrive - Stichting Deltares\Documents\Projects\Long involvement\SITO_Netwerk to system\project_files\get_shortest_routes_road')
study_area_suffix = '_GR'
output_name = f'study_area{study_area_suffix}'
polygon_path = root_folder.joinpath(f'static/network/polygon{study_area_suffix}.geojson')
od_file = root_folder.joinpath('static/network/od_nodes.geojson')

In [5]:
# Setting up the config data
config_data = NetworkConfigData(
    static_path=root_folder.joinpath('static'),
    crs=pyproj.CRS("EPSG:4326")
)

config_data.network = NetworkSection(
    network_type='drive',
    road_types=['motorway', 'motorway_link', 'primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary',
                'tertiary_link', 'residential'],
    polygon=polygon_path
)

In [5]:
# Download the graph from osm and save

osm_network_wrapper = OsmNetworkWrapper(config_data=config_data)
complex_graph = osm_network_wrapper.get_clean_graph_from_osm()
complex_graph.graph["crs"] = pyproj.CRS("EPSG:4326")
with open(root_folder.joinpath(f'static/output_graph/road_complex_graph_{output_name}.p'), 'wb') as f:
    pickle.dump(complex_graph, f)

c:\Users\asgarpou\AppData\Local\miniconda3\envs\ra2ce_multi_network_env\lib\site-packages\osmnx\simplification.py:459: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = utils_graph.graph_to_gdfs(G, edges=False)["geometry"].buffer(tolerance).unary_union
c:\Users\asgarpou\AppData\Local\miniconda3\envs\ra2ce_multi_network_env\lib\site-packages\osmnx\simplification.py:507: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid


In [6]:
# Map origin-destinations the complex graph

if not isinstance(complex_graph, MultiDiGraph):
    complex_graph = MultiDiGraph(complex_graph)

od_gdf, complex_graph = add_od_nodes(od=gpd.read_file(od_file), graph=complex_graph, crs=pyproj.CRS("EPSG:4326"))

Adding Origin-Destination nodes to graph: 5it [00:00, 34.34it/s]


In [7]:
# Save the outputs

od_gdf.to_file(od_file, driver="GeoJSON")

complex_graph_exporter = MultiGraphNetworkExporter(
    basename=f'road_complex_graph_origin_destinations_mapped_{output_name}',
    export_types=['pickle', 'shp'])
complex_graph_exporter.export_to_shp(output_dir=root_folder.joinpath(f'static/output_graph'),
                                     export_data=complex_graph)
complex_graph_exporter.export_to_pickle(output_dir=root_folder.joinpath(f'static/output_graph'),
                                        export_data=complex_graph)

ERROR:fiona._env:sqlite3_exec(PRAGMA journal_mode = OFF;
PRAGMA synchronous = OFF;
CREATE VIRTUAL TABLE my_rtree USING rtree(id, minx, maxx, miny, maxy)) failed: 
ERROR:fiona._env:sqlite3_exec(PRAGMA journal_mode = OFF;
PRAGMA synchronous = OFF;
CREATE VIRTUAL TABLE my_rtree USING rtree(id, minx, maxx, miny, maxy)) failed: 


In [6]:
# Load graphs and networks prepared befor
# Approach 1

od_gdf = gpd.read_file(od_file)

with open(root_folder.joinpath(f'static/output_graph/road_complex_graph_origin_destinations_mapped_{output_name}.p'), 'rb') as f:
    complex_graph = pickle.load(f)

complex_road_network = Network(
        nodes=_nx_to_network(complex_graph).nodes,
        edges=_nx_to_network(complex_graph).edges
    )

# Approach 2

# complex_road_network = Network(
#     nodes=gpd.read_file(root_folder.joinpath(f'static/output_graph/road_complex_graph_origin_destinations_mapped_{output_name}_nodes.gpkg')),
#     edges=gpd.read_file(root_folder.joinpath(f'static/output_graph/road_complex_graph_origin_destinations_mapped_{output_name}_edges.gpkg'))
# )

In [7]:
# Visualise

road_map = complex_road_network.edges.explore(tiles="CartoDB positron")
# road_map = od_gdf.explore(m=road_map, color='springgreen')
road_nodes_od_mapped = complex_road_network.nodes[~complex_road_network.nodes['od_id'].isna()]

In [8]:
road_nodes_od_mapped.explore(m=road_map, color='darkgreen')

: 